![titlecard](https://imgur.com/lw15BGH.png)

# DEVA: Tracking Anything with Decoupled Video Segmentation

[Ho Kei Cheng](https://hkchengrex.github.io/), [Seoung Wug Oh](https://sites.google.com/view/seoungwugoh/), [Brian Price](https://www.brianpricephd.com/), [Alexander Schwing](https://www.alexander-schwing.de/), [Joon-Young Lee](https://joonyoung-cv.github.io/)

University of Illinois Urbana-Champaign and Adobe

ICCV 2023

[[arXiv]](https://arxiv.org/abs/2309.03903) [[PDF]](https://arxiv.org/pdf/2309.03903.pdf) [[Project Page]](https://hkchengrex.github.io/Tracking-Anything-with-DEVA/) [[Github]](https://github.com/hkchengrex/Tracking-Anything-with-DEVA)

---

## Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model inference times.

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Wed Jun 12 11:11:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

# temporary fix for some weird locale bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

HOME: /content


## Install Grounding DINO and Segment Anything Model

We use grounded segment anything as our image model. It consists of two components: [Grounding DINO](https://github.com/IDEA-Research/GroundingDINO) - for zero-shot detection and [Segment Anything Model (SAM)](https://github.com/facebookresearch/segment-anything) - for converting boxes into segmentations. We have to install them first.

We install our fork of grounded segment anything which comes with a minor bug fix.


In [3]:
%cd {HOME}
!git clone https://github.com/hkchengrex/Grounded-Segment-Anything
%env CUDA_HOME=/usr/local/cuda
%env BUILD_WITH_CUDA=True
%env AM_I_DOCKER=False
%cd {HOME}/Grounded-Segment-Anything
!pip uninstall -y GroundingDINO
!pip install -e GroundingDINO
!pip install -q -e segment_anything

/content
Cloning into 'Grounded-Segment-Anything'...
remote: Enumerating objects: 1469, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1469 (delta 268), reused 259 (delta 259), pack-reused 1149
Receiving objects: 100% (1469/1469), 103.80 MiB | 18.22 MiB/s, done.
Resolving deltas: 100% (683/683), done.
Updating files: 100% (225/225), done.
env: CUDA_HOME=/usr/local/cuda
env: BUILD_WITH_CUDA=True
env: AM_I_DOCKER=False
/content/Grounded-Segment-Anything
Obtaining file:///content/Grounded-Segment-Anything/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 16.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_

<font color='red' size=16>Please restart: Runtime -> Restart Runtime</font>

## Make sure GroundingDINO has been installed properly.
## If this does not work, **Runtime -> Restart Runtime and try again**

In [1]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

# temporary fix for some weird locale bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

HOME: /content


In [2]:
# either one of these should work
%cd {HOME}
try:
  import groundingdino
  from groundingdino.util.inference import Model as GroundingDINOModel
except ImportError:
  import GroundingDINO
  from GroundingDINO.groundingdino.util.inference import Model as GroundingDINOModel

/content


Then we install [DEVA](https://github.com/hkchengrex/Tracking-Anything-with-DEVA).

In [3]:
%cd {HOME}
!git clone https://github.com/hkchengrex/Tracking-Anything-with-DEVA
%cd {HOME}/Tracking-Anything-with-DEVA
!pip install -q -e .

/content
Cloning into 'Tracking-Anything-with-DEVA'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 552 (delta 70), reused 66 (delta 51), pack-reused 436
Receiving objects: 100% (552/552), 1.09 MiB | 21.85 MiB/s, done.
Resolving deltas: 100% (297/297), done.
/content/Tracking-Anything-with-DEVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1

### Download Model Weights

We need a few model weights for all of these to work. Below is a simplified download script from DEVA.

In [4]:
%cd {HOME}/Tracking-Anything-with-DEVA
!wget -q -P ./saves/ https://github.com/hkchengrex/Tracking-Anything-with-DEVA/releases/download/v1.0/DEVA-propagation.pth
!wget -q -P ./saves/ https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
!wget -q -P ./saves/ https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!wget -q -P ./saves/ https://github.com/hkchengrex/Tracking-Anything-with-DEVA/releases/download/v1.0/GroundingDINO_SwinT_OGC.py

/content/Tracking-Anything-with-DEVA


## Initializing general stuff for DEVA and getting default parameters



In [5]:
%cd {HOME}/Tracking-Anything-with-DEVA

import os
from os import path
from argparse import ArgumentParser

import torch
import numpy as np

from deva.model.network import DEVA
from deva.inference.inference_core import DEVAInferenceCore
from deva.inference.result_utils import ResultSaver
from deva.inference.eval_args import add_common_eval_args, get_model_and_config
from deva.inference.demo_utils import flush_buffer
from deva.ext.ext_eval_args import add_ext_eval_args, add_text_default_args
from deva.ext.grounding_dino import get_grounding_dino_model
from deva.ext.with_text_processor import process_frame_with_text as process_frame

from tqdm import tqdm
import json

torch.autograd.set_grad_enabled(False)

# for id2rgb
np.random.seed(42)

# default parameters
parser = ArgumentParser()
add_common_eval_args(parser)
add_ext_eval_args(parser)
add_text_default_args(parser)

# load model and config
args = parser.parse_args([])
cfg = vars(args)
cfg['enable_long_term'] = True

# Load our checkpoint
deva_model = DEVA(cfg).cuda().eval()
if args.model is not None:
    model_weights = torch.load(args.model)
    deva_model.load_weights(model_weights)
else:
    print('No model loaded.')

gd_model, sam_model = get_grounding_dino_model(cfg, 'cuda')

/content/Tracking-Anything-with-DEVA


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 149MB/s]
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 146MB/s]


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

### Set hyperparameters

Default values should generally work fine. See https://github.com/hkchengrex/Tracking-Anything-with-DEVA/blob/main/docs/DEMO.md for some explanation of these parameters.

In [6]:
cfg['enable_long_term_count_usage'] = True
cfg['max_num_objects'] = 50
cfg['size'] = 480
cfg['DINO_THRESHOLD'] = 0.35
cfg['amp'] = True
cfg['chunk_size'] = 4
cfg['detection_every'] = 5
cfg['max_missed_detection_count'] = 10
cfg['sam_variant'] = 'original'
cfg['temporal_setting'] = 'online' # semionline usually works better; but online is faster for this demo
cfg['pluralize'] = True

## Download Example Data

Let's download an example video. Feel free to replace the video with yours. All you have to do is upload them to the `{HOME}/data` directory.

In [7]:
f"{HOME}/data"

'/content/data'

In [8]:
!mkdir -p {HOME}/data
%cd {HOME}/data

!wget -q -O example.mp4 https://user-images.githubusercontent.com/7107196/265518886-e5f6df87-9fd0-4178-8490-00c4b8dc613b.mp4

/content/data


## Specifying all the inputs and output directory

In [11]:
SOURCE_VIDEO_PATH = "/content/vech.mp4"
CLASSES = ['vehicles']
cfg['DINO_THRESHOLD'] = 0.5
OUTPUT_VIDEO_PATH = f"{HOME}/data/example_output.webm"

## Running DEVA

In [12]:
%cd {HOME}/Tracking-Anything-with-DEVA

from deva.ext.with_text_processor import process_frame_with_text as process_frame_text
import tempfile
import cv2

cfg['prompt'] = '.'.join(CLASSES)

deva = DEVAInferenceCore(deva_model, config=cfg)
deva.next_voting_frame = cfg['num_voting_frames'] - 1
deva.enabled_long_id()

# obtain temporary directory
result_saver = ResultSaver(None, None, dataset='gradio', object_manager=deva.object_manager)
writer_initizied = False

cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
ti = 0
# only an estimate
with torch.cuda.amp.autocast(enabled=cfg['amp']):
    with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))) as pbar:
        while (cap.isOpened()):
            ret, frame = cap.read()
            if ret == True:
                if not writer_initizied:
                    h, w = frame.shape[:2]
                    writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'vp80'), fps, (w, h))
                    writer_initizied = True
                    result_saver.writer = writer

                process_frame_text(deva,
                                    gd_model,
                                    sam_model,
                                    'null.png',
                                    result_saver,
                                    ti,
                                    image_np=frame)
                ti += 1
                pbar.update(1)
            else:
                break
    flush_buffer(deva, result_saver)
writer.release()
cap.release()
deva.clear_buffer()

/content/Tracking-Anything-with-DEVA


100%|██████████| 856/856 [09:17<00:00,  1.53it/s]


## Play the output video

In [12]:
from IPython.display import HTML
from base64 import b64encode
webm = open(OUTPUT_VIDEO_PATH,'rb').read()
data_url = "data:video/webm;base64," + b64encode(webm).decode()
HTML("""
<video width=720 controls>
      <source src="%s" type="video/webm">
</video>
""" % data_url)